# Create resaidual norm files for CREDIT

In [1]:
import os
import yaml
import copy
import numpy as np
import xarray as xr

In [2]:
from scipy.stats import gmean

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## File creation for physics project

In [4]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

N_levels = 37
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'

# get variable names
varnames = list(conf['residual'].keys())
varnames = varnames[:-5] # remove save_loc and others

varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']))
varname_upper = ['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']

In [5]:
# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['residual']['save_loc'] + '{}_mean_std_{}.npy'.format(
        conf['residual']['prefix'], varname)
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['residual']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(
            conf['residual']['prefix'], i_level, varname)
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

keys_to_drop = ['VAR_10U', 'VAR_10V', 'SP', 'Q', 'Q500', 'land_sea_CI_mask']
MEAN_values = {k: v for k, v in MEAN_values.items() if k not in keys_to_drop}
STD_values = {k: v for k, v in STD_values.items() if k not in keys_to_drop}

In [18]:
# separate upper air (list) and surf (float) std values
std_val_all = list(STD_values.values())
std_val_surf = np.array(std_val_all[:-5])
std_val_upper = std_val_all[-5:]

# combine and normalize
std_concat = np.concatenate([std_val_surf]+ std_val_upper)
std_g = gmean(np.sqrt(std_concat))
#std_g = np.mean(np.sqrt(std_concat))

In [13]:
ds_example = xr.open_zarr(base_dir+'all_in_one/ERA5_plevel_1deg_6h_1979_conserve.zarr')
level = np.array(ds_example['level'])

In [14]:
# ------------------------------------------------------- #
# create xr.DataArray for std
ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    #data = np.sqrt(data) / std_g # <--- var to std and divided by std_g
    data = data / std_g # <--- var to std and divided by std_g
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [16]:
# ds_std_6h.to_netcdf(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')

In [21]:
# ------------------------------------------------------- #
# Compare with my old ones
std_conserve = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')
std_bilinear = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_bilinear_1deg.nc')

for varname in list(std_conserve.keys()):
    print('=============== {} ================='.format(varname))
    print(np.array(std_conserve[varname]))
    print(np.array(std_bilinear[varname]))

=============== SKT =================
0.9020328939906355
0.8889482229414569
=============== surface_latent_heat_flux =================
2.756990226781677
2.7338782622675653
=============== top_net_solar_radiation =================
6.285933663515951
6.097818137560312
=============== VAR_2T =================
0.5812322200302392
0.5771408616318606
=============== surface_net_thermal_radiation =================
3.1111771870427525
3.082331817881059
=============== toa_incident_solar_radiation =================
6.302868904142561
6.115183631423283
=============== surface_sensible_heat_flux =================
5.077296641030351
4.921228881376689
=============== MSL =================
0.9710019153823931
0.947887088931082
=============== TCC =================
3.451359736171103
3.6788048004145764
=============== total_precipitation =================
4.418252724674436
4.625040749189226
=============== top_net_thermal_radiation =================
1.8856349420386707
1.9081347668682982
=============== evap

## File creation for comparison project

In [4]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

N_levels = 37
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'

# get variable names
varnames = list(conf['residual'].keys())
varnames = varnames[:-5] # remove save_loc and others

varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']))
varname_upper = ['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']

In [5]:
# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['residual']['save_loc'] + '{}_mean_std_{}.npy'.format(
        conf['residual']['prefix'], varname)
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['residual']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(
            conf['residual']['prefix'], i_level, varname)
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

keys_to_drop = ['TCC', 'SKT', 'SP', 'Q', 'land_sea_CI_mask']
MEAN_values = {k: v for k, v in MEAN_values.items() if k not in keys_to_drop}
STD_values = {k: v for k, v in STD_values.items() if k not in keys_to_drop}

In [6]:
# separate upper air (list) and surf (float) std values
std_val_all = list(STD_values.values())
std_val_surf = np.array(std_val_all[:-5])
std_val_upper = std_val_all[-5:]

# combine and normalize
std_concat = np.concatenate([std_val_surf]+ std_val_upper)
std_g = gmean(np.sqrt(std_concat))

In [7]:
ds_example = xr.open_zarr(base_dir+'all_in_one/ERA5_plevel_1deg_6h_1979_conserve.zarr')
level = np.array(ds_example['level'])

In [8]:
# ------------------------------------------------------- #
# create xr.DataArray for std
ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    data = np.sqrt(data) / std_g # <--- var to std and divided by std_g
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [10]:
ds_std_6h.to_netcdf(base_dir+'mean_std/residual_6h_1979_2019_1deg_project2.nc')

In [13]:
# ------------------------------------------------------- #
# Compare with my old ones
std_new = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_1deg_project2.nc')
std_conserve = xr.open_dataset(base_dir+'mean_std/residual_6h_1979_2019_conserve_1deg.nc')

for varname in list(std_conserve.keys()):
    try:
        print('=============== {} ================='.format(varname))
        print(np.array(std_new[varname]))
        print(np.array(std_conserve[varname]))
    except:
        pass

=============== SKT =================
=============== surface_latent_heat_flux =================
2.7513399703211547
2.756990226781677
=============== top_net_solar_radiation =================
6.273051087093417
6.285933663515951
=============== VAR_2T =================
0.5800410257073914
0.5812322200302392
=============== surface_net_thermal_radiation =================
3.104801049459763
3.1111771870427525
=============== toa_incident_solar_radiation =================
6.289951620142237
6.302868904142561
=============== surface_sensible_heat_flux =================
5.066891080695283
5.077296641030351
=============== MSL =================
0.9690119156383704
0.9710019153823931
=============== TCC =================
=============== total_precipitation =================
4.409197828230006
4.418252724674436
=============== top_net_thermal_radiation =================
1.8817704666009476
1.8856349420386707
=============== evaporation =================
2.7469841453482315
2.752625456520836
===========